In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Data Load

In [2]:
clean_vertederos = pd.read_csv('../data/Madrid/vertederos_clean.csv')
black_vertederos = pd.read_csv('../data/Madrid/puntos_negros.csv')

In [3]:
clean_vertederos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23510 entries, 0 to 23509
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place          23510 non-null  object 
 1   residue        23510 non-null  object 
 2   population     13422 non-null  float64
 3   near_nature    23509 non-null  float64
 4   near_river     23509 non-null  float64
 5   puntos_negros  23510 non-null  int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 1.1+ MB


# Data cleaning

In [4]:
black_vertederos_clean = black_vertederos.drop(['LATITUDE', 'LONGITUDE'], axis=1)

In [5]:
clean_vertederos_with_dummies = pd.get_dummies(clean_vertederos, columns=['residue'])

In [6]:
clean_vertederos_with_dummies

,place,population,near_nature,near_river,puntos_negros,residue_Madera y Corcho,residue_Metales y Aleaciones,residue_Papel y Cartón,residue_Plásticos y Caucho,residue_Residuos de Construcción y Demolición,residue_Sustancias Químicas Peligrosas y Materiales,residue_Vidrio
0,Alcalá de Henares,195000.0,0.0,0.0,0,False,True,False,False,False,False,False
1,Alcalá de Henares,195000.0,0.0,0.0,0,False,True,False,False,False,False,False
2,Alcalá de Henares,195000.0,0.0,0.0,0,False,True,False,False,False,False,False
3,Alcalá de Henares,195000.0,0.0,0.0,0,False,True,False,False,False,False,False
4,Alcalá de Henares,195000.0,0.0,0.0,0,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
23505,Madrid,3223000.0,0.0,0.0,0,False,True,False,False,False,False,False
23506,Madrid,3223000.0,0.0,0.0,0,False,False,False,False,False,True,False
23507,Madrid,3223000.0,0.0,0.0,0,False,True,False,False,False,False,False
23508,Madrid,3223000.0,0.0,0.0,0,False,True,False,False,False,False,False


In [7]:
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Madera y Corcho': 'Madera y Corcho'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Metales y Aleaciones': 'Metales y Aleaciones'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Papel y Cartón': 'Papel y Cartón'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Plásticos y Caucho': 'Plásticos y Caucho'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Residuos de Construcción y Demolición': 'Residuos de Construcción y Demolición'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Sustancias Químicas Peligrosas y Materiales': 'Sustancias Químicas Peligrosas y Materiales'})
clean_vertederos_with_dummies = clean_vertederos_with_dummies.rename(columns={'residue_Vidrio': 'Vidrio'})

In [8]:
clean_vertederos_with_dummies['near_nature'].unique()

array([ 0.,  1., nan])

In [9]:
clean_vertederos_with_dummies['near_nature'] = clean_vertederos_with_dummies['near_nature'] != 0

In [10]:
clean_vertederos_with_dummies['near_nature'].unique()

array([False,  True])

In [11]:
clean_vertederos_with_dummies['puntos_negros'] = clean_vertederos_with_dummies['puntos_negros'] != 0

In [12]:
clean_vertederos_with_dummies['puntos_negros'].unique()

array([False])

In [13]:
clean_vertederos_with_dummies['near_river'] = clean_vertederos_with_dummies['near_river'] != 0

In [14]:
clean_vertederos_with_dummies['near_river'].unique()

array([False,  True])

In [15]:
clean_vertederos_with_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23510 entries, 0 to 23509
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   place                                        23510 non-null  object 
 1   population                                   13422 non-null  float64
 2   near_nature                                  23510 non-null  bool   
 3   near_river                                   23510 non-null  bool   
 4   puntos_negros                                23510 non-null  bool   
 5   Madera y Corcho                              23510 non-null  bool   
 6   Metales y Aleaciones                         23510 non-null  bool   
 7   Papel y Cartón                               23510 non-null  bool   
 8   Plásticos y Caucho                           23510 non-null  bool   
 9   Residuos de Construcción y Demolición        23510 non-null  bool   
 10

In [16]:
black_vertederos_clean

,near_nature,near_river,place,Population,Residuos de Construcción y Demolición,Papel y Cartón,Plásticos y Caucho,Metales y Aleaciones,Sustancias Químicas Peligrosas y Materiales,Madera y Corcho,Vidrio,puntos_negros
0,1,1,Madrid,3340176,1,1,1,0,0,0,0,1
1,1,1,Leganés,190974,1,0,1,1,0,0,0,1
2,1,1,Alcorcón,172845,0,0,1,0,0,0,0,1
3,1,1,Paracuellos de Jarama,27034,1,0,0,0,0,0,0,1
4,1,1,Paracuellos de Jarama,27034,1,0,0,0,0,0,0,1
5,1,1,Madrid,3340176,1,0,0,0,0,0,0,1
6,1,1,Madrid,3340176,0,0,0,0,1,0,0,1
7,1,1,Brunete,10992,1,0,0,0,0,0,0,1
8,1,1,Daganzo de Arriba,10601,1,0,0,0,0,0,0,1
9,1,1,San Sebastián de los Reyes,92417,1,0,1,0,0,0,0,1


In [17]:
black_vertederos_clean['near_river'] = black_vertederos_clean['near_river'] != 0

In [18]:
black_vertederos_clean['near_nature'] = black_vertederos_clean['near_nature'] != 0
black_vertederos_clean['Residuos de Construcción y Demolición'] = black_vertederos_clean['Residuos de Construcción y Demolición'] != 0
black_vertederos_clean['Papel y Cartón'] = black_vertederos_clean['Papel y Cartón'] != 0
black_vertederos_clean['Plásticos y Caucho'] = black_vertederos_clean['Plásticos y Caucho'] != 0

In [19]:
black_vertederos_clean

,near_nature,near_river,place,Population,Residuos de Construcción y Demolición,Papel y Cartón,Plásticos y Caucho,Metales y Aleaciones,Sustancias Químicas Peligrosas y Materiales,Madera y Corcho,Vidrio,puntos_negros
0,True,True,Madrid,3340176,True,True,True,0,0,0,0,1
1,True,True,Leganés,190974,True,False,True,1,0,0,0,1
2,True,True,Alcorcón,172845,False,False,True,0,0,0,0,1
3,True,True,Paracuellos de Jarama,27034,True,False,False,0,0,0,0,1
4,True,True,Paracuellos de Jarama,27034,True,False,False,0,0,0,0,1
5,True,True,Madrid,3340176,True,False,False,0,0,0,0,1
6,True,True,Madrid,3340176,False,False,False,0,1,0,0,1
7,True,True,Brunete,10992,True,False,False,0,0,0,0,1
8,True,True,Daganzo de Arriba,10601,True,False,False,0,0,0,0,1
9,True,True,San Sebastián de los Reyes,92417,True,False,True,0,0,0,0,1


In [20]:
black_vertederos_clean.columns

Index(['near_nature', 'near_river', 'place', 'Population',
       'Residuos de Construcción y Demolición', 'Papel y Cartón',
       'Plásticos y Caucho', 'Metales y Aleaciones',
       'Sustancias Químicas Peligrosas y Materiales', 'Madera y Corcho',
       'Vidrio', 'puntos_negros'],
      dtype='object')

In [21]:
columns_to_convert = [
    'Metales y Aleaciones',
    'Sustancias Químicas Peligrosas y Materiales',
    'Madera y Corcho',
    'Vidrio',
    'puntos_negros'
]

for column in columns_to_convert:
    black_vertederos_clean[column] = black_vertederos_clean[column].astype(bool)


In [22]:
black_vertederos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   near_nature                                  37 non-null     bool  
 1   near_river                                   37 non-null     bool  
 2   place                                        37 non-null     object
 3   Population                                   37 non-null     int64 
 4   Residuos de Construcción y Demolición        37 non-null     bool  
 5   Papel y Cartón                               37 non-null     bool  
 6   Plásticos y Caucho                           37 non-null     bool  
 7   Metales y Aleaciones                         37 non-null     bool  
 8   Sustancias Químicas Peligrosas y Materiales  37 non-null     bool  
 9   Madera y Corcho                              37 non-null     bool  
 10  Vidrio          

In [23]:
# Define the new order of columns as they appear in clean_vertederos_with_dummies
new_column_order = [
    'place', 
    'Population',  # Note: Assuming you want to match 'Population' with 'population' from the other DataFrame
    'near_nature', 
    'near_river', 
    'puntos_negros', 
    'Madera y Corcho', 
    'Metales y Aleaciones', 
    'Papel y Cartón', 
    'Plásticos y Caucho', 
    'Residuos de Construcción y Demolición', 
    'Sustancias Químicas Peligrosas y Materiales', 
    'Vidrio'
]

# Reorder the columns in black_vertederos_clean
black_vertederos_clean = black_vertederos_clean[new_column_order]

# If 'Population' in black_vertederos_clean matches 'population' in dtype (int64 vs. float64), 
# and you intend to perform operations considering them as equivalent, you may need to adjust types accordingly
# For instance, converting 'Population' in black_vertederos_clean to float to match 'population' in clean_vertederos_with_dummies
# black_vertederos_clean['Population'] = black_vertederos_clean['Population'].astype(float)

# Check the reordered DataFrame
black_vertederos_clean.head()


,place,Population,near_nature,near_river,puntos_negros,Madera y Corcho,Metales y Aleaciones,Papel y Cartón,Plásticos y Caucho,Residuos de Construcción y Demolición,Sustancias Químicas Peligrosas y Materiales,Vidrio
0,Madrid,3340176,True,True,True,False,False,True,True,True,False,False
1,Leganés,190974,True,True,True,False,True,False,True,True,False,False
2,Alcorcón,172845,True,True,True,False,False,False,True,False,False,False
3,Paracuellos de Jarama,27034,True,True,True,False,False,False,False,True,False,False
4,Paracuellos de Jarama,27034,True,True,True,False,False,False,False,True,False,False


In [24]:
black_vertederos_clean = black_vertederos_clean.rename(columns={'Population': 'population'})
black_vertederos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   place                                        37 non-null     object
 1   population                                   37 non-null     int64 
 2   near_nature                                  37 non-null     bool  
 3   near_river                                   37 non-null     bool  
 4   puntos_negros                                37 non-null     bool  
 5   Madera y Corcho                              37 non-null     bool  
 6   Metales y Aleaciones                         37 non-null     bool  
 7   Papel y Cartón                               37 non-null     bool  
 8   Plásticos y Caucho                           37 non-null     bool  
 9   Residuos de Construcción y Demolición        37 non-null     bool  
 10  Sustancias Quími

In [25]:
clean_vertederos_with_dummies['population'] = clean_vertederos_with_dummies['population'].astype('Int64')

In [26]:
clean_vertederos_with_dummies.info()
clean_vertederos_ok = clean_vertederos_with_dummies.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23510 entries, 0 to 23509
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   place                                        23510 non-null  object
 1   population                                   13422 non-null  Int64 
 2   near_nature                                  23510 non-null  bool  
 3   near_river                                   23510 non-null  bool  
 4   puntos_negros                                23510 non-null  bool  
 5   Madera y Corcho                              23510 non-null  bool  
 6   Metales y Aleaciones                         23510 non-null  bool  
 7   Papel y Cartón                               23510 non-null  bool  
 8   Plásticos y Caucho                           23510 non-null  bool  
 9   Residuos de Construcción y Demolición        23510 non-null  bool  
 10  Sustancias

In [27]:
clean_vertederos_ok = clean_vertederos_with_dummies.dropna()

In [28]:
black_vertederos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   place                                        37 non-null     object
 1   population                                   37 non-null     int64 
 2   near_nature                                  37 non-null     bool  
 3   near_river                                   37 non-null     bool  
 4   puntos_negros                                37 non-null     bool  
 5   Madera y Corcho                              37 non-null     bool  
 6   Metales y Aleaciones                         37 non-null     bool  
 7   Papel y Cartón                               37 non-null     bool  
 8   Plásticos y Caucho                           37 non-null     bool  
 9   Residuos de Construcción y Demolición        37 non-null     bool  
 10  Sustancias Quími

# Data Combination

In [29]:
combined_df = pd.concat([black_vertederos_clean, clean_vertederos_ok], ignore_index=True)
combined_df 

,place,population,near_nature,near_river,puntos_negros,Madera y Corcho,Metales y Aleaciones,Papel y Cartón,Plásticos y Caucho,Residuos de Construcción y Demolición,Sustancias Químicas Peligrosas y Materiales,Vidrio
0,Madrid,3340176,True,True,True,False,False,True,True,True,False,False
1,Leganés,190974,True,True,True,False,True,False,True,True,False,False
2,Alcorcón,172845,True,True,True,False,False,False,True,False,False,False
3,Paracuellos de Jarama,27034,True,True,True,False,False,False,False,True,False,False
4,Paracuellos de Jarama,27034,True,True,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
13454,Madrid,3223000,False,False,False,False,True,False,False,False,False,False
13455,Madrid,3223000,False,False,False,False,False,False,False,False,True,False
13456,Madrid,3223000,False,False,False,False,True,False,False,False,False,False
13457,Madrid,3223000,False,False,False,False,True,False,False,False,False,False


In [30]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 12 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   place                                        13459 non-null  object
 1   population                                   13459 non-null  Int64 
 2   near_nature                                  13459 non-null  bool  
 3   near_river                                   13459 non-null  bool  
 4   puntos_negros                                13459 non-null  bool  
 5   Madera y Corcho                              13459 non-null  bool  
 6   Metales y Aleaciones                         13459 non-null  bool  
 7   Papel y Cartón                               13459 non-null  bool  
 8   Plásticos y Caucho                           13459 non-null  bool  
 9   Residuos de Construcción y Demolición        13459 non-null  bool  
 10  Sustancias

In [31]:
X = combined_df.drop(['puntos_negros','place'], axis=1)  # Dropping 'place' for simplicity, and 'puntos_negros' is the target
y = combined_df['puntos_negros']

In [32]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 10 columns):
 #   Column                                       Non-Null Count  Dtype
---  ------                                       --------------  -----
 0   population                                   13459 non-null  Int64
 1   near_nature                                  13459 non-null  bool 
 2   near_river                                   13459 non-null  bool 
 3   Madera y Corcho                              13459 non-null  bool 
 4   Metales y Aleaciones                         13459 non-null  bool 
 5   Papel y Cartón                               13459 non-null  bool 
 6   Plásticos y Caucho                           13459 non-null  bool 
 7   Residuos de Construcción y Demolición        13459 non-null  bool 
 8   Sustancias Químicas Peligrosas y Materiales  13459 non-null  bool 
 9   Vidrio                                       13459 non-null  bool 
dtypes: Int64(1), bool(9)
m

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost Model

In [34]:

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [35]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [36]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9985141158989599
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      2682
        True       1.00      0.60      0.75        10

    accuracy                           1.00      2692
   macro avg       1.00      0.80      0.87      2692
weighted avg       1.00      1.00      1.00      2692



# Logistic Regression

In [37]:
model = LogisticRegression(solver='liblinear', random_state=0).fit(X, y)

In [38]:
model.classes_

array([False,  True])

In [39]:
model.intercept_

array([-1.44060447])

In [40]:
 model.coef_

array([[-1.80756120e-05, -1.79607595e-01, -1.78883625e-01,
        -5.40065824e-02, -2.51036399e-01, -4.14295378e-02,
        -7.43312113e-02,  6.90105048e-02, -1.04552770e+00,
        -3.17554899e-02]])

In [41]:
model.predict_proba(X)

array([[1.00000000e+00, 9.49543756e-28],
       [9.95959848e-01, 4.04015166e-03],
       [9.93291881e-01, 6.70811850e-03],
       ...,
       [1.00000000e+00, 9.21156081e-27],
       [1.00000000e+00, 9.21156081e-27],
       [1.00000000e+00, 4.16188460e-27]])

In [42]:
 modellogic = model.predict(X)

In [43]:
modellogic

array([False, False, False, ..., False, False, False])